In [ ]:
%pip install mediapipe

In [ ]:
%pip install pandas

In [ ]:
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
from PIL import Image
import os

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

def process_video(input_path, output_path):
    """
    Process a video with Mediapipe Pose + Hands.
    Draws skeleton and hand landmarks, saves to output_path.
    """
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"❌ Could not open {input_path}")
        return

    # Setup output video writer
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    pose = mp_pose.Pose(static_image_mode=False, model_complexity=2)
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(image_rgb)
        hand_results = hands.process(image_rgb)
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Draw pose
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image_bgr,
                pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

        # Draw hands
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image_bgr,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
                )

        out.write(image_bgr)

    pose.close()
    hands.close()
    cap.release()
    out.release()
    print(f"✅ Processed and saved: {output_path}")

def process_multiple_videos(input_folder, output_folder):
    """
    Runs Mediapipe processing on all videos in a folder.
    Saves outputs to output_folder with same filenames.
    """
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"processed_{filename}")
            process_video(input_path, output_path)

# Example usage:
if __name__ == "__main__":
    input_folder = "vids"           # your folder containing raw videos
    output_folder = "landmarked"  # folder for processed outputs
    process_multiple_videos(input_folder, output_folder)


In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

# landmark indices from mediapipe pose model
FACE_POINTS = {
    "nose": 0,
    "left_eye": 2,
    "right_eye": 5,
    "mouth_left": 9,
    "mouth_right": 10
}

def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"❌ Could not open {input_path}")
        return

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    pose = mp_pose.Pose(
        static_image_mode=False,
        model_complexity=2,
        smooth_landmarks=True,
        min_detection_confidence=0.3,
        min_tracking_confidence=0.3
    )
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.3,
        min_tracking_confidence=0.3
    )

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(image_rgb)
        hand_results = hands.process(image_rgb)
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        
        frame_landmarks = []

        # ----- Pose landmarks (33 points) -----
        if pose_results.pose_landmarks:
            for lm in pose_results.pose_landmarks.landmark:
                frame_landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
        else:
            frame_landmarks.extend([0.0] * (33*4))  # pad if not detected

        # ----- Minimal face points (5 points) -----
        if pose_results.pose_landmarks:
            for idx in FACE_POINTS.values():
                lm = pose_results.pose_landmarks.landmark[idx]
                frame_landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
        else:
            frame_landmarks.extend([0.0] * (5*4))

        # ----- Hands (21 landmarks per hand, 2 hands) -----
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                for lm in hand_landmarks.landmark:
                    frame_landmarks.extend([lm.x, lm.y, lm.z, 1.0])
        # if no hands detected, pad for 2 hands
        if not hand_results.multi_hand_landmarks:
            frame_landmarks.extend([0.0] * (21*2*4))

        # Add to overall list
        all_frames.append(frame_landmarks)

        
        
        
        
        # Draw pose landmarks (body)
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image_bgr,
                pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

            # Draw minimal face points (eyes, nose, mouth)
            h, w, _ = image_bgr.shape
            for name, idx in FACE_POINTS.items():
                lm = pose_results.pose_landmarks.landmark[idx]
                x, y = int(lm.x * w), int(lm.y * h)
                cv2.circle(image_bgr, (x, y), 5, (0, 255, 255), -1)
                # cv2.putText(image_bgr, name, (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

        # Draw hands
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image_bgr,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
                )

        out.write(image_bgr)

    pose.close()
    hands.close()
    cap.release()
    out.release()
    print(f"✅ Processed and saved: {output_path}")

def process_multiple_videos(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    # ip = ["00335.mp4","00"]
    
    
    # return
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"processed_{filename}")
            process_video(input_path, output_path)

# Example usage
if __name__ == "__main__":
    input_folder = "vids"           # your folder containing raw videos
    output_folder = "landmarked"  # folder for processed outputs
    process_multiple_videos(input_folder, output_folder)


In [1]:
import cv2
import mediapipe as mp
import os

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

# minimal face points from Pose model
FACE_POINTS = [0, 2, 5, 9, 10]

def play_videos_loop(input_folder):
    pose = mp_pose.Pose(
        static_image_mode=False,
        model_complexity=2,
        smooth_landmarks=True,
        min_detection_confidence=0.3,
        min_tracking_confidence=0.3
    )
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.3,
        min_tracking_confidence=0.3
    )

    video_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))]
    if not video_files:
        print("❌ No video files found")
        return

    print("🎥 Playing videos. Press Ctrl+C in terminal to stop.")

    try:
        while True:  # infinite loop
            for filename in video_files:
                cap = cv2.VideoCapture(os.path.join(input_folder, filename))
                if not cap.isOpened():
                    print(f"❌ Could not open {filename}")
                    continue

                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break

                    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    pose_results = pose.process(image_rgb)
                    hand_results = hands.process(image_rgb)
                    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

                    # Pose landmarks
                    if pose_results.pose_landmarks:
                        mp_drawing.draw_landmarks(
                            image_bgr,
                            pose_results.pose_landmarks,
                            mp_pose.POSE_CONNECTIONS,
                            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
                        )
                        # minimal face points
                        h, w, _ = image_bgr.shape
                        for idx in FACE_POINTS:
                            lm = pose_results.pose_landmarks.landmark[idx]
                            x, y = int(lm.x * w), int(lm.y * h)
                            cv2.circle(image_bgr, (x, y), 5, (0, 255, 255), -1)

                    # Hands
                    if hand_results.multi_hand_landmarks:
                        for hand_landmarks in hand_results.multi_hand_landmarks:
                            mp_drawing.draw_landmarks(
                                image_bgr,
                                hand_landmarks,
                                mp_hands.HAND_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
                            )

                    cv2.imshow("Landmarks Viewer", image_bgr)
                    if cv2.waitKey(1) & 0xFF == 27:  # ESC key to skip video
                        break

                cap.release()
    except KeyboardInterrupt:
        print("\n🛑 Stopped by user.")

    pose.close()
    hands.close()
    cv2.destroyAllWindows()


# Example usage
if __name__ == "__main__":
    play_videos_loop("vids")


🎥 Playing videos. Press Ctrl+C in terminal to stop.

🛑 Stopped by user.


In [ ]:
# TESTTTTT FULL


import cv2
import mediapipe as mp
import numpy as np
import os

mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

# minimal face landmark indices from pose model
FACE_POINTS = [0, 2, 5, 9, 10]

def extract_landmarks_from_video(video_path):
    pose = mp_pose.Pose(
        static_image_mode=False,
        model_complexity=2,
        smooth_landmarks=True,
        min_detection_confidence=0.3,
        min_tracking_confidence=0.3
    )
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.3,
        min_tracking_confidence=0.3
    )

    cap = cv2.VideoCapture(video_path)
    all_frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(image_rgb)
        hand_results = hands.process(image_rgb)

        frame_data = []

        # ----- Pose landmarks (torso + limbs) -----
        if pose_results.pose_landmarks:
            for lm in pose_results.pose_landmarks.landmark:
                frame_data.extend([lm.x, lm.y, lm.z, lm.visibility])
        else:
            frame_data.extend([0.0] * (33 * 4))  # fill missing with zeros

        # ----- Minimal face points -----
        if pose_results.pose_landmarks:
            for idx in FACE_POINTS:
                lm = pose_results.pose_landmarks.landmark[idx]
                frame_data.extend([lm.x, lm.y, lm.z, lm.visibility])
        else:
            frame_data.extend([0.0] * (len(FACE_POINTS) * 4))

        # ----- Hands -----
        for hand_landmarks in (hand_results.multi_hand_landmarks or []):
            for lm in hand_landmarks.landmark:
                frame_data.extend([lm.x, lm.y, lm.z, 1.0])
        # if no hand detected, pad for 2 hands (21 landmarks each)
        if not hand_results.multi_hand_landmarks:
            frame_data.extend([0.0] * (21 * 2 * 4))

        all_frames.append(frame_data)

    cap.release()
    pose.close()
    hands.close()

    return np.array(all_frames, dtype=np.float32)


def process_videos_to_numpy(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename.replace('.', '_') + ".npy")

            landmarks = extract_landmarks_from_video(input_path)
            np.save(output_path, landmarks)
            print(f"✅ Saved {output_path} with shape {landmarks.shape}")


# Example usage
if __name__ == "__main__":
    process_videos_to_numpy("videos", "landmark_data")


In [ ]:
import cv2
import mediapipe as mp
import os
import numpy as np

# Mediapipe setup
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

# Minimal face points from pose landmarks
FACE_POINTS = {
    "nose": 0,
    "left_eye": 2,
    "right_eye": 5,
    "mouth_left": 9,
    "mouth_right": 10
}

def process_video(input_path, output_folder):
    # Open video
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"❌ Could not open {input_path}")
        return

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)

    # Prepare output video path
    base_name = os.path.basename(input_path)
    video_out_path = os.path.join(output_folder, f"processed_{base_name}")
    npy_out_path   = os.path.join(output_folder, f"{os.path.splitext(base_name)[0]}.npy")

    out = cv2.VideoWriter(video_out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    # Initialize Mediapipe models
    pose = mp_pose.Pose(
        static_image_mode=False,
        model_complexity=2,
        smooth_landmarks=True,
        min_detection_confidence=0.3,
        min_tracking_confidence=0.3
    )
    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.3,
        min_tracking_confidence=0.3
    )

    all_frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR → RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(image_rgb)
        hand_results = hands.process(image_rgb)
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Draw pose landmarks
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image_bgr,
                pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

            # Draw minimal face points
            h, w, _ = image_bgr.shape
            for idx in FACE_POINTS.values():
                lm = pose_results.pose_landmarks.landmark[idx]
                x, y = int(lm.x * w), int(lm.y * h)
                cv2.circle(image_bgr, (x, y), 5, (0, 255, 255), -1)

        # Draw hands
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image_bgr,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
                )

        # ----- Extract landmarks for ML -----
        frame_landmarks = []

        # Pose landmarks (33)
        if pose_results.pose_landmarks:
            for lm in pose_results.pose_landmarks.landmark:
                frame_landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
        else:
            frame_landmarks.extend([0.0]*(33*4))

        # Minimal face points (5)
        if pose_results.pose_landmarks:
            for idx in FACE_POINTS.values():
                lm = pose_results.pose_landmarks.landmark[idx]
                frame_landmarks.extend([lm.x, lm.y, lm.z, lm.visibility])
        else:
            frame_landmarks.extend([0.0]*(5*4))

        # Hands (2 hands x 21 landmarks)
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                for lm in hand_landmarks.landmark:
                    frame_landmarks.extend([lm.x, lm.y, lm.z, 1.0])
        # pad if no hands detected
        if not hand_results.multi_hand_landmarks:
            frame_landmarks.extend([0.0]*(21*2*4))

        all_frames.append(frame_landmarks)

        # Write frame to output video
        out.write(image_bgr)

    # Cleanup
    pose.close()
    hands.close()
    cap.release()
    out.release()

    # Save landmarks as numpy file
    all_frames_np = np.array(all_frames, dtype=np.float32)
    np.save(npy_out_path, all_frames_np)

    print(f"✅ Processed: {video_out_path}")
    print(f"✅ Landmarks saved: {npy_out_path} (shape={all_frames_np.shape})")


def process_multiple_videos(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):
            input_path = os.path.join(input_folder, filename)
            process_video(input_path, output_folder)


# Example usage
if __name__ == "__main__":
    input_folder = "vids"           # folder with raw videos
    output_folder = "landmarked"    # folder for videos + landmarks
    process_multiple_videos(input_folder, output_folder)
